Let's run the python profiler to see what the issue is

In [1]:
import lsst.sims.skybrightness as sb
import profile
import numpy as np
import healpy as hp

In [2]:
sm = sb.SkyModel(mags=True)
nside = 16
hpmap = np.zeros(hp.nside2npix(nside))
lat, ra = hp.pix2ang(nside, np.arange(hpmap.size))
dec = np.pi/2-lat
good = np.where(dec < 0)
dec = dec[good]
ra = ra[good]
print 'computing at %i points on the sky' % ra.size

computing at 1504 points on the sky


In [6]:
mjd = 56948.020683
profile_results = profile.run('sm.setRaDecMjd(ra,dec,mjd)')

         30456 function calls (30319 primitive calls) in 1.087 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.000    0.000 :0(__new__)
       20    0.000    0.000    0.002    0.000 :0(__reduce_ex__)
        1    0.000    0.000    0.000    0.000 :0(_dtdb)
        1    0.000    0.000    0.000    0.000 :0(_gc2gd)
        1    0.000    0.000    0.000    0.000 :0(_gd2gc)
        1    0.000    0.000    0.000    0.000 :0(_taitt)
        4    0.000    0.000    0.000    0.000 :0(_taiutc)
        1    0.000    0.000    0.000    0.000 :0(_tttai)
        1    0.000    0.000    0.000    0.000 :0(_tttdb)
        1    0.000    0.000    0.000    0.000 :0(_utcut1)
       10    0.000    0.000    0.000    0.000 :0(abs)
       32    0.000    0.000    0.000    0.000 :0(all)
        1    0.001    0.001    0.001    0.001 :0(altazVector)
       34    0.000    0.000    0.000    0.000 :0(any)
        1    0.000    0

In [4]:
profile.run('sm.returnMags()')

         32 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 :0(decode)
        1    0.000    0.000    0.000    0.000 :0(endswith)
        1    0.000    0.000    0.000    0.000 :0(getpid)
        4    0.000    0.000    0.000    0.000 :0(isinstance)
        1    0.000    0.000    0.000    0.000 :0(issubclass)
        1    0.000    0.000    0.000    0.000 :0(len)
        1    0.000    0.000    0.000    0.000 :0(open)
        1    0.000    0.000    0.000    0.000 :0(readlines)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 :0(startswith)
        1    0.000    0.000    0.000    0.000 :0(stat)
        1    0.000    0.000    0.000    0.000 :0(strip)
        1    0.000    0.000    0.000    0.000 :0(time)
        1    0.000    0.000    0.000    0.000 :0(utf_8_decode)
        1    0.000    0.000    0.000  

/Users/yoachim/gitRepos/sims_skybrightness/python/lsst/sims/skybrightness/skyModel.py:336: RuntimeWarning: divide by zero encountered in log10
  mags = -2.5*np.log10(self.spec)+np.log10(3631.)


In [5]:
# Looks like the alt az transform is killing me!
from lsst.sims.utils import haversine, _raDecFromAltAz, _altAzPaFromRaDec, Site, ObservationMetaData
telescope =  Site('LSST')
omd = ObservationMetaData(mjd=mjd,site=telescope)
altaz_profile = profile.run('_altAzPaFromRaDec(ra,dec,omd)')

         5823 function calls (5687 primitive calls) in 0.952 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.000    0.000    0.000    0.000 :0(__new__)
       20    0.000    0.000    0.001    0.000 :0(__reduce_ex__)
        1    0.000    0.000    0.000    0.000 :0(_dtdb)
        1    0.000    0.000    0.000    0.000 :0(_gc2gd)
        1    0.000    0.000    0.000    0.000 :0(_gd2gc)
        1    0.000    0.000    0.000    0.000 :0(_taitt)
        4    0.000    0.000    0.000    0.000 :0(_taiutc)
        1    0.000    0.000    0.000    0.000 :0(_tttai)
        1    0.000    0.000    0.000    0.000 :0(_tttdb)
        1    0.000    0.000    0.000    0.000 :0(_utcut1)
       15    0.000    0.000    0.000    0.000 :0(all)
        1    0.001    0.001    0.001    0.001 :0(altazVector)
       32    0.000    0.000    0.000    0.000 :0(any)
        1    0.000    0.000    0.000    0.000 :0(aoppa)
        1    0.923    0